In [17]:
import math
import pandas as pd
import nltk as nltk
import math
import operator
nltk.download('punkt')


[nltk_data] Downloading package punkt to /home/manorlf/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [18]:
df = pd.read_csv("./data/estadao_noticias_eleicao.csv")
df = df.fillna('')
df

,timestamp,titulo,subTitulo,conteudo,url,idNoticia
0,2014-12-31T00:00:00Z,PT espera 30 mil pessoas em festa na Esplanada,Objetivo é demonstrar apoio popular a Dilma e ...,BRASÍLIA - Após o desgaste provocado com o lan...,"http://politica.estadao.com.br/noticias/geral,...",1
1,2014-12-31T00:00:00Z,Alckmin toma posse de olho no Planalto,Governador reeleito tenta amarrar tucanos paul...,"Reeleito em outubro, o governador tucano Geral...","http://politica.estadao.com.br/noticias/geral,...",2
2,2014-12-31T00:00:00Z,Seis obstáculos e desafios do segundo mandato ...,"Em meio a escândalo de corrupção, presidente t...",1. Rearranjo das contas A nova equipe econôm...,"http://politica.estadao.com.br/noticias/geral,...",3
3,2014-12-31T00:00:00Z,,Veja as principais fotos do dia e dos eventos ...,,"http://fotos.estadao.com.br/fotos/politica,dil...",4
4,2014-12-31T00:00:00Z,,Veja as principais fotos do dia e dos eventos ...,,"http://fotos.estadao.com.br/fotos/politica,dil...",5
5,2014-12-31T00:00:00Z,,Veja as principais fotos do dia e dos eventos ...,,"http://fotos.estadao.com.br/fotos/politica,dil...",6
6,2014-12-31T00:00:00Z,Veja os desafios dos governadores que assumem ...,,"No Acre, governador reeleito quer erradicar an...","http://politica.estadao.com.br/noticias/geral,...",7
7,2014-12-31T00:00:00Z,PT impulsiona cerimônia de posse da Dilma nas ...,"Já entre os 27 governadores eleitos, apenas se...","Os perfis da presidente Dilma Rousseff, nas re...","http://politica.estadao.com.br/noticias/geral,...",8
8,2014-12-31T00:00:00Z,,"Em setembro de 2014, após conceder entrevista ...",,"http://tv.estadao.com.br/videos,politica,dilma...",9
9,2014-12-31T00:00:00Z,Dilma veta projeto de Suplicy que cria linha o...,Proposta havia sido aprovada no Senado depois ...,A presidente Dilma Rousseff vetou integralment...,"http://politica.estadao.com.br/noticias/geral,...",10


<h1>Vector Sapace Retrieval Model</h1>

<p>Binary Representation</p>

In [19]:
terms_set = dict() # {term : [(doc_id:tf)]}

#if it's a binary model, the tf will be 0 or 1
#otherwise the term frequence in the doc

def fill_terms_set(term, data):
    if (term in terms_set):
        result = terms_set[term]
    else:
        terms_set[term] = []
        rows = data.shape[0]
    
        for doc in range(rows):
            tf = 0
            title = (data.loc[doc, 'titulo']).lower()
            subTitle = (data.loc[doc, 'subTitulo']).lower()
            content = (data.loc[doc, 'conteudo']).lower()
        
            text = nltk.word_tokenize(title + ' ' + subTitle + ' ' + content)
            i = 0
            while (i < len(text)):
                if (text[i] == term.lower()):
                    tf += 1
                i += 1
            if (tf > 0):
                terms_set[term].append((data.loc[doc, 'idNoticia'],tf))
            
        result = terms_set[term]
        
    return result

def idf(word, data):
    total_docs = data.shape[0]
    k = len(terms_set[word])
    calc_idf = math.log((total_docs - 1)/k)
    return calc_idf

In [20]:

def find_intersect(term1, term2): #term1 =  [(doc_1: 2), (doc_2, 1)]
    result = []                                  #term2 =  [(doc_1: 2), (doc_2, 0)]
    n_term1 = []
    n_term2 = []
    i = 0
    j = 0
    
    while (i < len(term1) and j < len(term2)):
        if (term1[i][0] == term2[j][0]):
            if (term1[i][1] and term2[j][1]):
                result.append(term1[i])
                result.append(term2[j])

            i += 1
            j += 1
        elif (term1[i][0] < term2[j][0]):
            i += 1
        else:
            j += 1
    
    return n_term1, n_term2


def sum_tf_intersect(term1, term2): #term1 =  [(doc_1: 2), (doc_2, 1)]
    result = []
    i = 0
    j = 0
    aux = (0,0)
    
    while (i < len(term1) and j < len(term2)):
        if (term1[i][0] == term2[j][0]):
            if (term1[i][1] and term2[j][1]):
                aux = (term1[i][0], term1[i][1] + term2[j][1])
                result.append(aux)
            i += 1
            j += 1
        elif (term1[i][0] < term2[j][0]):
            i += 1
        else:
            j += 1

    
    return result


In [112]:
fill_terms_set("segundo", "binary", df)


[(1, 1, 2),
 (2, 1, 1),
 (3, 1, 4),
 (7, 1, 22),
 (8, 1, 2),
 (11, 1, 2),
 (13, 1, 1),
 (14, 1, 1),
 (16, 1, 1),
 (18, 1, 1),
 (19, 1, 2),
 (23, 1, 3),
 (25, 1, 1),
 (26, 1, 2),
 (28, 1, 1),
 (29, 1, 1),
 (31, 1, 4),
 (33, 1, 1),
 (35, 1, 6),
 (36, 1, 2),
 (38, 1, 2),
 (41, 1, 2),
 (43, 1, 3),
 (45, 1, 1),
 (46, 1, 3),
 (47, 1, 2),
 (51, 1, 2),
 (52, 1, 4),
 (54, 1, 1),
 (56, 1, 2),
 (57, 1, 1),
 (58, 1, 2),
 (59, 1, 2),
 (60, 1, 1),
 (62, 1, 3),
 (64, 1, 1),
 (69, 1, 5),
 (71, 1, 3),
 (72, 1, 2),
 (74, 1, 7),
 (75, 1, 2),
 (76, 1, 2),
 (77, 1, 1),
 (78, 1, 2),
 (79, 1, 1),
 (80, 1, 3),
 (81, 1, 4),
 (82, 1, 2),
 (83, 1, 1),
 (85, 1, 1),
 (86, 1, 2),
 (87, 1, 1),
 (88, 1, 3),
 (89, 1, 2),
 (91, 1, 2),
 (92, 1, 3),
 (95, 1, 4),
 (98, 1, 2),
 (99, 1, 1),
 (102, 1, 1),
 (104, 1, 2),
 (105, 1, 1),
 (106, 1, 2),
 (107, 1, 3),
 (108, 1, 4),
 (111, 1, 1),
 (113, 1, 2),
 (114, 1, 2),
 (116, 1, 1),
 (117, 1, 5),
 (118, 1, 3),
 (119, 1, 4),
 (121, 1, 1),
 (123, 1, 3),
 (124, 1, 1),
 (125, 1, 1),

In [21]:
#Example
# q = "lava jato"
# lava: [(doc_1, 3), (doc_2, 4), (doc_3, 5)]
# jato: [(doc_2: 2), (doc_3, 3)]

# needs to be in the same docs
# doc_2   doc_3
#   4       5    tf_lava
#   2       3    tf_jato
#   6       8    Result

def populate_terms_set(query, data):
    q = query.split()
    for term in q:
        fill_terms_set(term, data)


In [22]:
def vsm_tf(query, data):
    populate_terms_set(query,data)
    doc_results = []
    q = query.split()
    aux = terms_set[q[0]]
    for i in range(1, len(q)):
        doc_results += sum_tf_intersect(aux, terms_set[q[i]])     
        aux = doc_results
    return doc_results   



In [10]:
def vsm_binary(query, data):
    populate_terms_set(query,data)
    doc_results = []
    q = query.split()
    
    return doc_results 

In [123]:
result = vsm_binary('segundo turno', df)
result.sort(key=operator.itemgetter(1), reverse=True)
print(result[:5])

[(7, 22), (2744, 18), (2388, 13), (2112, 12), (7672, 12)]


<p>TF</p>

In [122]:
result = vsm_tf('segundo turno', df)
result.sort(key=operator.itemgetter(1), reverse=True)
print(result[:5])

[(2744, 43), (7, 28), (2112, 26), (7672, 26), (2388, 24)]


<p>TF-IDF</p>

In [25]:
# how to calculate IDF
# idf(w) = log((n - 1)/k)
# where n is the total of documents and
# k is the total of documents where ocurs the word w
# TF-IDF we will multiply the idf for tf of each word

def idf(word, data):
    total_docs = data.shape[0]
    k = len(terms_set[word])
    calc_idf = math.log((total_docs - 1)/k)
    return calc_idf

def idf_term(term, data):
    populate_terms_set(term, data)
    cont = 0
    result = 0
    for doc in terms_set[term]:
        cont += 1
    return math.log((data.shape[0] + 1)/cont)

def vsm_tf_idf(query, data):
    q = query.split()
    populate_terms_set(query,data)
    cal_idf = 0
    result = 0
   
    for term in q:
        cal_idf = idf_term(term, data)
        for doc in terms_set[term]:
            result += doc[1] * cal_idf
    return result
    

            
    
    


In [26]:
result = vsm_tf_idf("segundo turno", df)
result.sort(key=operator.itemgetter(1), reverse=True)
print(result[:5])

AttributeError: 'float' object has no attribute 'sort'